In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import * 
from pyspark.sql.functions import lit
import pyodbc
import pandas as pd
import pyspark.sql.functions as sf


In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "15g").getOrCreate()

In [3]:
path = "C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\Dataset\\"

In [4]:
file = "20220401.json"

In [5]:
df = spark.read.json(path+file)

In [6]:
df = df.select( '_source.Contract','_source.AppName', '_source.TotalDuration')

In [7]:
df.show()

+---------+-------+-------------+
| Contract|AppName|TotalDuration|
+---------+-------+-------------+
|HNH579912|  KPLUS|          254|
|HUFD40665|  KPLUS|         1457|
|HNH572635|  KPLUS|         2318|
|HND141717|  KPLUS|         1452|
|HNH743103|  KPLUS|          251|
|HNH893773|  KPLUS|          924|
|HND083642|  KPLUS|         1444|
|DNFD74404|  KPLUS|          691|
|DTFD21200|  KPLUS|         1436|
|LDFD05747|  KPLUS|         1434|
|HNH063566|  KPLUS|          687|
|HNH866786|  KPLUS|          248|
|NBAAA1128|  KPLUS|          247|
|HNH960439|  KPLUS|          683|
|HNJ035736|  KPLUS|          246|
|NTFD93673|  KPLUS|         2288|
|HNJ063267|  KPLUS|         2282|
|HNH790383|  KPLUS|          906|
|THFD12466|  KPLUS|          242|
|HNH566080|  KPLUS|          242|
+---------+-------+-------------+
only showing top 20 rows



In [8]:
for i in range(2,10):
    file = "2022040{}.json".format(i)
    df1 = spark.read.json(path+file)
    df1 = df1.select( '_source.Contract','_source.AppName', '_source.TotalDuration')
    df = df.union(df1)
    

In [11]:
df.count()

14566752

In [9]:
for i in range(10,31):
    file = "202204{}.json".format(i)
    df1 = spark.read.json(path+file)
    df1 = df1.select( '_source.Contract','_source.AppName', '_source.TotalDuration')
    df = df.union(df1)
    

In [13]:
df.count()

48457499

In [10]:
df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- AppName: string (nullable = true)
 |-- TotalDuration: long (nullable = true)



In [14]:
df.show()

+---------+-------+-------------+
| Contract|AppName|TotalDuration|
+---------+-------+-------------+
|HNH579912|  KPLUS|          254|
|HUFD40665|  KPLUS|         1457|
|HNH572635|  KPLUS|         2318|
|HND141717|  KPLUS|         1452|
|HNH743103|  KPLUS|          251|
|HNH893773|  KPLUS|          924|
|HND083642|  KPLUS|         1444|
|DNFD74404|  KPLUS|          691|
|DTFD21200|  KPLUS|         1436|
|LDFD05747|  KPLUS|         1434|
|HNH063566|  KPLUS|          687|
|HNH866786|  KPLUS|          248|
|NBAAA1128|  KPLUS|          247|
|HNH960439|  KPLUS|          683|
|HNJ035736|  KPLUS|          246|
|NTFD93673|  KPLUS|         2288|
|HNJ063267|  KPLUS|         2282|
|HNH790383|  KPLUS|          906|
|THFD12466|  KPLUS|          242|
|HNH566080|  KPLUS|          242|
+---------+-------+-------------+
only showing top 20 rows



In [11]:
df = df.withColumn("Type",
       when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
      .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
             (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
      .when((col("AppName") == 'RELAX'), "Giải Trí")
      .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
      .when((col("AppName") == 'SPORT'), "Thể Thao")
      .otherwise("Error"))

In [12]:
df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- AppName: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Type: string (nullable = false)



In [13]:
df = df.drop('AppName')

In [14]:
df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Type: string (nullable = false)



In [19]:
df.orderBy('Contract').show()

+--------+-------------+-----------+
|Contract|TotalDuration|       Type|
+--------+-------------+-----------+
|        |           52|Truyền Hình|
|        |         3041|Truyền Hình|
|        |           35|Truyền Hình|
|        |          125|      Error|
|        |           84|Truyền Hình|
|        |         4969|Truyền Hình|
|       0|          188|Truyền Hình|
|       0|          184|Truyền Hình|
|       0|          185|Truyền Hình|
|       0|         9527|Truyền Hình|
|       0|          175|Truyền Hình|
|       0|          185|Truyền Hình|
|       0|         9403|Truyền Hình|
|       0|          188|Truyền Hình|
|       0|         9283|Truyền Hình|
|       0|        14863|Truyền Hình|
|       0|          186|Truyền Hình|
|       0|        23177|Truyền Hình|
|       0|         9251|Truyền Hình|
|       0|          199|Truyền Hình|
+--------+-------------+-----------+
only showing top 20 rows



In [15]:
df = df.filter((df.Contract !='') & (df.Contract != '0')).orderBy('Contract')

In [16]:
df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Type: string (nullable = false)



In [30]:
df.show()

+--------------+-------------+-----------+
|      Contract|TotalDuration|       Type|
+--------------+-------------+-----------+
|113.182.209.48|           89|   Giải Trí|
|113.182.209.48|           20|Truyền Hình|
|113.182.209.48|           43|Truyền Hình|
|113.183.15.246|         8203|Truyền Hình|
| 113.183.60.85|         1191|Truyền Hình|
|113.191.153.52|         9497|Truyền Hình|
|116.107.44.221|          193|Truyền Hình|
|116.107.44.221|           57|Phim Truyện|
|116.107.44.221|           74|   Giải Trí|
| 123.17.50.134|          293|Truyền Hình|
| 123.25.79.212|         9922|Truyền Hình|
| 123.25.79.212|         3361|Truyền Hình|
|14.166.200.230|           14|Truyền Hình|
|14.181.179.168|         3142|Truyền Hình|
|14.182.110.125|           92|   Giải Trí|
|14.182.110.125|          404|Truyền Hình|
| 14.182.41.177|           29|Truyền Hình|
|14.189.124.168|          147|Truyền Hình|
|14.189.216.181|         1345|Truyền Hình|
| 14.191.177.89|          164|Truyền Hình|
+----------

In [17]:
df = df.filter(df.Type != 'Error')

In [18]:
df.cache()

DataFrame[Contract: string, TotalDuration: bigint, Type: string]

In [33]:
df.show()

+--------------+-------------+-----------+
|      Contract|TotalDuration|       Type|
+--------------+-------------+-----------+
|113.182.209.48|           20|Truyền Hình|
|113.182.209.48|           43|Truyền Hình|
|113.182.209.48|           89|   Giải Trí|
|113.183.15.246|         8203|Truyền Hình|
| 113.183.60.85|         1191|Truyền Hình|
|113.191.153.52|         9497|Truyền Hình|
|116.107.44.221|           57|Phim Truyện|
|116.107.44.221|          193|Truyền Hình|
|116.107.44.221|           74|   Giải Trí|
| 123.17.50.134|          293|Truyền Hình|
| 123.25.79.212|         3361|Truyền Hình|
| 123.25.79.212|         9922|Truyền Hình|
|14.166.200.230|           14|Truyền Hình|
|14.181.179.168|         3142|Truyền Hình|
|14.182.110.125|          404|Truyền Hình|
|14.182.110.125|           92|   Giải Trí|
| 14.182.41.177|           29|Truyền Hình|
|14.189.124.168|          147|Truyền Hình|
|14.189.216.181|         1345|Truyền Hình|
| 14.191.177.89|          164|Truyền Hình|
+----------

In [34]:
df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- TotalDuration: long (nullable = true)
 |-- Type: string (nullable = false)



In [38]:
#df.withColumn('TotalDuration',df.TotalDuration.cast(FloatType())).show()

In [46]:
#df.groupBy('Contract','Type').sum('TotalDuration').orderBy('Contract').show()

+---------------+-----------+------------------+
|       Contract|       Type|sum(TotalDuration)|
+---------------+-----------+------------------+
| 113.182.209.48|Truyền Hình|                63|
| 113.182.209.48|   Giải Trí|                89|
| 113.183.15.246|Truyền Hình|              8203|
|  113.183.60.85|Truyền Hình|              1191|
| 113.191.153.52|Truyền Hình|              9497|
| 116.107.44.221|Phim Truyện|                57|
| 116.107.44.221|Truyền Hình|               193|
| 116.107.44.221|   Giải Trí|                74|
|  123.17.50.134|Truyền Hình|               293|
|  123.25.79.212|Truyền Hình|             13283|
| 14.166.200.230|Truyền Hình|                14|
| 14.181.179.168|Truyền Hình|              3142|
| 14.182.110.125|Truyền Hình|               404|
| 14.182.110.125|   Giải Trí|                92|
|  14.182.41.177|Truyền Hình|                29|
| 14.189.124.168|Truyền Hình|               147|
| 14.189.216.181|Truyền Hình|              1345|
|  14.191.177.89|Tru

In [19]:
df = df.groupBy('Contract','Type').sum('TotalDuration').orderBy('Contract')

In [20]:
df.printSchema()

root
 |-- Contract: string (nullable = true)
 |-- Type: string (nullable = false)
 |-- sum(TotalDuration): long (nullable = true)



In [48]:
df.count()

2729276

In [21]:
df.withColumn('Date',lit('2022-04-01')).show()

+---------------+-----------+------------------+----------+
|       Contract|       Type|sum(TotalDuration)|      Date|
+---------------+-----------+------------------+----------+
| 113.182.209.48|   Giải Trí|                89|2022-04-01|
| 113.182.209.48|Truyền Hình|                63|2022-04-01|
| 113.183.15.246|Truyền Hình|              8203|2022-04-01|
|  113.183.60.85|Truyền Hình|              1191|2022-04-01|
| 113.191.153.52|Truyền Hình|              9497|2022-04-01|
| 116.107.44.221|   Giải Trí|                74|2022-04-01|
| 116.107.44.221|Truyền Hình|               193|2022-04-01|
| 116.107.44.221|Phim Truyện|                57|2022-04-01|
|  123.17.50.134|Truyền Hình|               293|2022-04-01|
|  123.25.79.212|Truyền Hình|             13283|2022-04-01|
| 14.166.200.230|Truyền Hình|                14|2022-04-01|
| 14.181.179.168|Truyền Hình|              3142|2022-04-01|
| 14.182.110.125|   Giải Trí|                92|2022-04-01|
| 14.182.110.125|Truyền Hình|           

In [22]:
df = df.withColumn('Date',lit('2022-04-01'))

In [23]:
df = df.withColumnRenamed('sum(TotalDuration)','TotalDuration')

In [59]:
df.select('Type').distinct().show()

+-----------+
|       Type|
+-----------+
|   Thể Thao|
|Truyền Hình|
|  Thiếu Nhi|
|   Giải Trí|
|Phim Truyện|
+-----------+



In [24]:
thethao = df.filter(df.Type == 'Thể Thao')
truyenhinh = df.filter(df.Type == 'Truyền Hình')
thieunhi = df.filter(df.Type == 'Thiếu Nhi' )
giaitri = df.filter(df.Type == 'Giải Trí')
phimtruyen = df.filter(df.Type == 'Phim Truyện')

In [25]:
thethao = thethao.withColumnRenamed('TotalDuration','SportDuration').drop('Type')
truyenhinh = truyenhinh.withColumnRenamed('TotalDuration','TVDuration').drop('Type')
thieunhi = thieunhi.withColumnRenamed('TotalDuration','ChildDuration').drop('Type')
giaitri = giaitri.withColumnRenamed('TotalDuration','RelaxDuration').drop('Type')
phimtruyen = phimtruyen.withColumnRenamed('TotalDuration','MovieDuration').drop('Type')

In [26]:
thethao.show()

+---------+-------------+----------+
| Contract|SportDuration|      Date|
+---------+-------------+----------+
|AGAAA0674|          560|2022-04-01|
|AGAAA0681|           37|2022-04-01|
|AGAAA0932|         1967|2022-04-01|
|AGAAA1306|           29|2022-04-01|
|AGAAA1395|          243|2022-04-01|
|AGAAA1893|           76|2022-04-01|
|AGAAA1984|            0|2022-04-01|
|AGAAA1998|        33865|2022-04-01|
|AGAAA2047|           35|2022-04-01|
|AGAAA2051|           50|2022-04-01|
|AGAAA2188|           44|2022-04-01|
|AGAAA2262|           60|2022-04-01|
|AGAAA2534|          587|2022-04-01|
|AGAAA2624|          127|2022-04-01|
|AGAAA2657|           20|2022-04-01|
|AGAAA2661|           20|2022-04-01|
|AGAAA2729|         1943|2022-04-01|
|AGAAA2737|          502|2022-04-01|
|AGAAA2797|         2258|2022-04-01|
|AGAAA2834|          542|2022-04-01|
+---------+-------------+----------+
only showing top 20 rows



In [27]:
truyenhinh.show()

+---------------+----------+----------+
|       Contract|TVDuration|      Date|
+---------------+----------+----------+
| 113.182.209.48|        63|2022-04-01|
| 113.183.15.246|      8203|2022-04-01|
|  113.183.60.85|      1191|2022-04-01|
| 113.191.153.52|      9497|2022-04-01|
| 116.107.44.221|       193|2022-04-01|
|  123.17.50.134|       293|2022-04-01|
|  123.25.79.212|     13283|2022-04-01|
| 14.166.200.230|        14|2022-04-01|
| 14.181.179.168|      3142|2022-04-01|
| 14.182.110.125|       404|2022-04-01|
|  14.182.41.177|        29|2022-04-01|
| 14.189.124.168|       147|2022-04-01|
| 14.189.216.181|      1345|2022-04-01|
|  14.191.177.89|       164|2022-04-01|
|171.244.225.105|       105|2022-04-01|
| 222.252.254.74|       358|2022-04-01|
|      AGAAA0335|    232322|2022-04-01|
|      AGAAA0338|    278633|2022-04-01|
|      AGAAA0342|    117788|2022-04-01|
|      AGAAA0345|    352136|2022-04-01|
+---------------+----------+----------+
only showing top 20 rows



In [28]:
result = thethao.join(truyenhinh,['Contract','Date'],'outer').join(thieunhi,['Contract','Date'],'outer').join(giaitri,['Contract','Date'],'outer').join(phimtruyen,['Contract','Date'],'outer')

In [29]:
result.count()

1920544

In [30]:
result.show()

+---------+----------+-------------+----------+-------------+-------------+-------------+
| Contract|      Date|SportDuration|TVDuration|ChildDuration|RelaxDuration|MovieDuration|
+---------+----------+-------------+----------+-------------+-------------+-------------+
|AGAAA0520|2022-04-01|         null|    173458|         null|         null|          447|
|AGAAA0663|2022-04-01|         null|     89001|         null|         null|         null|
|AGAAA1058|2022-04-01|         null|    118784|         null|         null|         null|
|AGAAA1238|2022-04-01|         null|    163158|         null|         null|         null|
|AGAAA1572|2022-04-01|         null|     27309|         null|         null|         null|
|AGAAA1727|2022-04-01|         null|      5898|         null|         null|         null|
|AGAAA3240|2022-04-01|         null|     46748|           71|         null|          140|
|AGAAA3322|2022-04-01|         null|     42844|         null|         null|         1410|
|AGAAA3368

In [86]:
#result.repartition(1).write.csv('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 5\\Result',header='True')

In [31]:
df.show()

+---------------+-----------+-------------+----------+
|       Contract|       Type|TotalDuration|      Date|
+---------------+-----------+-------------+----------+
| 113.182.209.48|   Giải Trí|           89|2022-04-01|
| 113.182.209.48|Truyền Hình|           63|2022-04-01|
| 113.183.15.246|Truyền Hình|         8203|2022-04-01|
|  113.183.60.85|Truyền Hình|         1191|2022-04-01|
| 113.191.153.52|Truyền Hình|         9497|2022-04-01|
| 116.107.44.221|Truyền Hình|          193|2022-04-01|
| 116.107.44.221|   Giải Trí|           74|2022-04-01|
| 116.107.44.221|Phim Truyện|           57|2022-04-01|
|  123.17.50.134|Truyền Hình|          293|2022-04-01|
|  123.25.79.212|Truyền Hình|        13283|2022-04-01|
| 14.166.200.230|Truyền Hình|           14|2022-04-01|
| 14.181.179.168|Truyền Hình|         3142|2022-04-01|
| 14.182.110.125|Truyền Hình|          404|2022-04-01|
| 14.182.110.125|   Giải Trí|           92|2022-04-01|
|  14.182.41.177|Truyền Hình|           29|2022-04-01|
| 14.189.1

In [32]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import col

In [33]:
window = Window.partitionBy("Contract").orderBy(col('TotalDuration').desc())

In [34]:
rank_result = df.withColumn('RANK',rank().over(window))

In [35]:
rank_result.filter(rank_result.Contract=='AGAAA0520').show()

+---------+-----------+-------------+----------+----+
| Contract|       Type|TotalDuration|      Date|RANK|
+---------+-----------+-------------+----------+----+
|AGAAA0520|Truyền Hình|       173458|2022-04-01|   1|
|AGAAA0520|Phim Truyện|          447|2022-04-01|   2|
+---------+-----------+-------------+----------+----+



In [36]:
most_watched = rank_result.filter(rank_result.RANK == '1')

In [38]:
most_watched.show()

+---------------+-----------+-------------+----------+----+
|       Contract|       Type|TotalDuration|      Date|RANK|
+---------------+-----------+-------------+----------+----+
| 113.182.209.48|   Giải Trí|           89|2022-04-01|   1|
| 113.183.15.246|Truyền Hình|         8203|2022-04-01|   1|
|  113.183.60.85|Truyền Hình|         1191|2022-04-01|   1|
| 113.191.153.52|Truyền Hình|         9497|2022-04-01|   1|
| 116.107.44.221|Truyền Hình|          193|2022-04-01|   1|
|  123.17.50.134|Truyền Hình|          293|2022-04-01|   1|
|  123.25.79.212|Truyền Hình|        13283|2022-04-01|   1|
| 14.166.200.230|Truyền Hình|           14|2022-04-01|   1|
| 14.181.179.168|Truyền Hình|         3142|2022-04-01|   1|
| 14.182.110.125|Truyền Hình|          404|2022-04-01|   1|
|  14.182.41.177|Truyền Hình|           29|2022-04-01|   1|
| 14.189.124.168|Truyền Hình|          147|2022-04-01|   1|
| 14.189.216.181|Truyền Hình|         1345|2022-04-01|   1|
|  14.191.177.89|Truyền Hình|          1

In [39]:
most_watched = most_watched.drop('TotalDuration').drop('RANK').withColumnRenamed('Type','Most_Watch')

In [40]:
habbit = result.join(most_watched,['Contract','Date'],'inner')

In [41]:
habbit.show()

+---------+----------+-------------+----------+-------------+-------------+-------------+-----------+
| Contract|      Date|SportDuration|TVDuration|ChildDuration|RelaxDuration|MovieDuration| Most_Watch|
+---------+----------+-------------+----------+-------------+-------------+-------------+-----------+
|AGAAA0520|2022-04-01|         null|    173458|         null|         null|          447|Truyền Hình|
|AGAAA0663|2022-04-01|         null|     89001|         null|         null|         null|Truyền Hình|
|AGAAA1058|2022-04-01|         null|    118784|         null|         null|         null|Truyền Hình|
|AGAAA1238|2022-04-01|         null|    163158|         null|         null|         null|Truyền Hình|
|AGAAA1572|2022-04-01|         null|     27309|         null|         null|         null|Truyền Hình|
|AGAAA1727|2022-04-01|         null|      5898|         null|         null|         null|Truyền Hình|
|AGAAA3240|2022-04-01|         null|     46748|           71|         null|       

In [42]:
habbit.groupby("Most_Watch").count().show()

+-----------+-------+
| Most_Watch|  count|
+-----------+-------+
|   Thể Thao|    131|
|Truyền Hình|1865823|
|  Thiếu Nhi|  10913|
|   Giải Trí|    903|
|Phim Truyện|  42792|
+-----------+-------+



In [123]:
#habbit.repartition(1).write.csv('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 5\\HabbitResult',header='True')

In [44]:
#habbit = spark.read.csv('C:\\Users\\ASUS\\OneDrive\\Big_Data_Analytics\\BigData_Gen1\\Class 5\\HabbitResult\*.csv',header=True)

In [45]:
habbit.show()

+---------+----------+-------------+----------+-------------+-------------+-------------+-----------+
| Contract|      Date|SportDuration|TVDuration|ChildDuration|RelaxDuration|MovieDuration| Most_Watch|
+---------+----------+-------------+----------+-------------+-------------+-------------+-----------+
|AGAAA0520|2022-04-01|         null|    173458|         null|         null|          447|Truyền Hình|
|AGAAA0663|2022-04-01|         null|     89001|         null|         null|         null|Truyền Hình|
|AGAAA1058|2022-04-01|         null|    118784|         null|         null|         null|Truyền Hình|
|AGAAA1238|2022-04-01|         null|    163158|         null|         null|         null|Truyền Hình|
|AGAAA1572|2022-04-01|         null|     27309|         null|         null|         null|Truyền Hình|
|AGAAA1727|2022-04-01|         null|      5898|         null|         null|         null|Truyền Hình|
|AGAAA3240|2022-04-01|         null|     46748|           71|         null|       

In [48]:
#habbit.count()

In [47]:
#habbit = habbit.withColumn('Date',lit('2022-04-01'))

In [46]:
#habbit.show()

<h1> Import to Database <h1> 

In [49]:
output = habbit.toPandas()

In [50]:
output

,Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch
0,AGAAA0520,2022-04-01,None,173458,None,None,447,Truyền Hình
1,AGAAA0663,2022-04-01,None,89001,None,None,None,Truyền Hình
2,AGAAA1058,2022-04-01,None,118784,None,None,None,Truyền Hình
3,AGAAA1238,2022-04-01,None,163158,None,None,None,Truyền Hình
4,AGAAA1572,2022-04-01,None,27309,None,None,None,Truyền Hình
...,...,...,...,...,...,...,...,...
1920557,YBFD10364,2022-04-01,None,150370,None,None,None,Truyền Hình
1920558,YBFD10951,2022-04-01,None,86400,None,None,None,Truyền Hình
1920559,YBFD11137,2022-04-01,None,197844,None,None,None,Truyền Hình
1920560,YBFD11335,2022-04-01,None,758843,None,None,None,Truyền Hình


In [51]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920562 entries, 0 to 1920561
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Contract       object
 1   Date           object
 2   SportDuration  object
 3   TVDuration     object
 4   ChildDuration  object
 5   RelaxDuration  object
 6   MovieDuration  object
 7   Most_Watch     object
dtypes: object(8)
memory usage: 117.2+ MB


In [128]:
import mysql.connector
import pandas as pd

In [129]:
user = 'root'
password =''
host = 'localhost'
database = 'Data_Engineer'

In [130]:
cnx = mysql.connector.connect(user=user,password=password,host=host,database=database)

In [131]:
pd.read_sql("select * from customer_behaviour", cnx)

,Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,MostWatch


In [132]:
cursor = cnx.cursor()

In [133]:
cursor.execute("select * from customer_behaviour")

In [134]:
cursor.fetchall()

[]

In [109]:
output.columns

Index(['Contract', 'Date', 'SportDuration', 'TVDuration', 'ChildDuration',
       'RelaxDuration', 'MovieDuration', 'Most_Watch'],
      dtype='object')

In [141]:
def import_to_mysql(user,password,host,database,data):
    data = data[['Contract', 'Date', 'SportDuration', 'TVDuration', 'ChildDuration',
       'RelaxDuration', 'MovieDuration', 'Most_Watch']]
    cnx = mysql.connector.connect(user=user,password=password,host=host,database=database)
    cursor = cnx.cursor()
    sql = """INSERT INTO customer_behaviour (
        Contract,
        Date,
        SportDuration,
        TVDuration,
        ChildDuration,
        RelaxDuration,
        MovieDuration,
        MostWatch) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""
    cursor.executemany(sql,data.values.tolist())
    cnx.commit()
    return print("Import Successfully")

In [111]:
output = output.where(pd.notnull(output), None)

In [112]:
output.values.tolist()

[['AGAAA0520', '2022-04-01', None, '173458', None, None, '447', 'Truyền Hình'],
 ['AGAAA0663', '2022-04-01', None, '89001', None, None, None, 'Truyền Hình'],
 ['AGAAA1058', '2022-04-01', None, '118784', None, None, None, 'Truyền Hình'],
 ['AGAAA1238', '2022-04-01', None, '163158', None, None, None, 'Truyền Hình'],
 ['AGAAA1572', '2022-04-01', None, '27309', None, None, None, 'Truyền Hình'],
 ['AGAAA1727', '2022-04-01', None, '5898', None, None, None, 'Truyền Hình'],
 ['AGAAA3240', '2022-04-01', None, '46748', '71', None, '140', 'Truyền Hình'],
 ['AGAAA3322', '2022-04-01', None, '42844', None, None, '1410', 'Truyền Hình'],
 ['AGAAA3368', '2022-04-01', None, '24735', None, None, None, 'Truyền Hình'],
 ['AGAAA3654', '2022-04-01', None, '53982', None, None, None, 'Truyền Hình'],
 ['AGAAA3659', '2022-04-01', None, '18210', None, None, None, 'Truyền Hình'],
 ['AGD006240', '2022-04-01', None, '175415', '71', '97', '76', 'Truyền Hình'],
 ['AGD009631', '2022-04-01', None, '1199573', None, None,

In [113]:
sql = """Insert into customer360 (Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch) Values (%s,%s,%s,%s,%s,%s,%s,%s)"""

In [114]:
sql

'Insert into customer360 (Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch) Values (%s,%s,%s,%s,%s,%s,%s,%s)'

In [136]:
test = output.loc[0:10000]

In [137]:
test

,Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch
0,AGAAA0520,2022-04-01,None,173458,None,None,447,Truyền Hình
1,AGAAA0663,2022-04-01,None,89001,None,None,None,Truyền Hình
2,AGAAA1058,2022-04-01,None,118784,None,None,None,Truyền Hình
3,AGAAA1238,2022-04-01,None,163158,None,None,None,Truyền Hình
4,AGAAA1572,2022-04-01,None,27309,None,None,None,Truyền Hình
...,...,...,...,...,...,...,...,...
9996,BDFD36066,2022-04-01,None,246808,1528,None,None,Truyền Hình
9997,BDFD36548,2022-04-01,None,270006,6317,None,211009,Truyền Hình
9998,BDFD39166,2022-04-01,None,63685,668,None,None,Truyền Hình
9999,BDFD40868,2022-04-01,None,348768,None,None,None,Truyền Hình


In [138]:
test.values.tolist()[0]

['AGAAA0520', '2022-04-01', None, '173458', None, None, '447', 'Truyền Hình']

In [139]:
"""Insert into customer360 (Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch) Values (%s,%s,%s,%s,%s,%s,%s,%s)""" % tuple(test.values.tolist()[0])

'Insert into customer360 (Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,Most_Watch) Values (AGAAA0520,2022-04-01,None,173458,None,None,447,Truyền Hình)'

In [140]:
cnx = mysql.connector.connect(user=user,password=password,host=host,database=database)
cursor = cnx.cursor()
sql = """Insert into customer_behaviour (Contract,Date,SportDuration,TVDuration,ChildDuration,RelaxDuration,MovieDuration,MostWatch) Values (%s,%s,%s,%s,%s,%s,%s,%s)"""
cursor.executemany(sql,test.values.tolist())
cnx.commit()
print("Import Successfully")

Import Successfully


In [144]:
import_to_mysql(user,password,host,database,test)

Import Successfully


In [ ]:
url = 'jdbc:mysql://' + host + ':' + port + '/' + db_name
driver = "com.mysql.cj.jdbc.Driver"
data.write.format('jdbc').option('url',url).option('driver', driver).option('dbtable','events').option('user',user).option('password',password).mode('append').save()